**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
from PIL import Image
from sklearn.cluster import MiniBatchKMeans
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
DATA_HOME = "https://github.com/michalgregor/ml_notebooks/blob/main/data/{}?raw=1"

from class_utils.download import download_file_maybe_extract
download_file_maybe_extract(DATA_HOME.format("images/photo_rome.jpg"), directory="data")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

In [ ]:
#@title -- Auxiliary Functions -- { display-mode: "form" }

def plot_colors(colors, cluster_centers=None):
    _, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(14, 6))
    ax1.scatter(colors[:, 0], colors[:, 1], s=10, c=colors/255.0)

    ax1.set_xlabel("red")
    ax1.set_ylabel("green")
    ax1.grid(ls='--')
    ax1.set_axisbelow(True)

    if not cluster_centers is None:
        ax1.scatter(cluster_centers[:, 0], cluster_centers[:, 1], s=100,
            c='orange', edgecolors='k', linewidths=2.5
        )

    ax2.scatter(colors[:, 0], colors[:, 2], s=10, c=colors/255.0)
    ax2.set_xlabel("red")
    ax2.set_ylabel("blue")
    ax2.grid(ls='--')
    ax2.set_axisbelow(True)

    if not cluster_centers is None:
        ax2.scatter(cluster_centers[:, 0], cluster_centers[:, 2], s=100,
            c='orange', edgecolors='k', linewidths=2.5
        )

    ax3.scatter(colors[:, 1], colors[:, 2], s=10, c=colors/255.0)
    ax3.set_xlabel("green")
    ax3.set_ylabel("blue")
    ax3.grid(ls='--')
    ax3.set_axisbelow(True)

    if not cluster_centers is None:
        ax3.scatter(cluster_centers[:, 1], cluster_centers[:, 2], s=100,
            c='orange', edgecolors='k', linewidths=2.5
        )

## k-Means pre kvantizáciu farieb

V rámci tohto príkladu budeme aplikovať $k$-means na trochu odlišnú úlohu. Vezmeme si obrázok a budeme sa ho snažiť skomprimovať kvantizáciou priestoru farieb. Na začiatku bude náš obrázok v RGB priestore. Budeme pracovať s troma 8-bitovými číslami: s jedným pre každý farebný kanál (červený, modrý, zelený). Tým pádom budeme mať 256 úrovní pre každý farebný kanál, t.j. $256^3 = 16\ 777\ 216$ rôznych farieb.

Povedzme, že by sme namiesto toho ukladali spolu s obrázkom malú paletu farieb a pre každý pixel uložili len index farby v tejto palete. Potom by nám stačilo použiť len jedno 8-bitové číslo pre každý pixel namiesto troch. Pochopiteľne, toto číslo nemusí byť práve 8-bitové: mohlo by byť aj menšie alebo väčšie podľa toho aká veľká bude naša paleta.

V každom prípade, princíp je dosť jednoduchý – skutočnou otázkou je, ako nájsť dobrú paletu. Chceme do nej zahrnúť také farby, aby komprimovaný obrázok nebol priveľmi skreslený. Čo teda urobíme, bude, že na pixeloch z nášho obrázka vykonáme $k$-means zhlukovanie. Tým spôsobom získame paletu s $k$ farbami, ktoré budú reprezentovať zhluky v priestore farieb.

### Načítanie obrázka

Začnime tým, že si načítame a zobrazíme obrázok. Ako uvidíme, obsahuje prevažne zelené, fialové, modré, hnedé a biele farby.



In [ ]:
img = np.array(Image.open("data/photo_rome.jpg"))
plt.figure(figsize=(10, 6))
plt.imshow(img)
plt.axis('off');

### Zmena tvaru obrázka

Teraz zmeníme tvar obrázka tak, aby sme získali maticu bodov z priestoru farieb, t.j. maticu $m \times n$, kde $m$ je celkový počet pixelov z obrázka a $n$ je rozmer farebného priestoru – v našom prípade $n=3$ keďže náš obrázok je v RGB.



In [ ]:
img_shape = img.shape
X = img.reshape(-1, img_shape[2])
X.shape

### Explorácia priestoru farieb

Následne môžeme použiť `np.unique`, aby sme skontrolovali, koľko rôznych farieb obrázok obsahuje – ako vidno, je ich v skutočnosti dosť veľa.



In [ ]:
colors = np.unique(X, axis=0)
len(colors)

Aby sme lepšie vizuálne porozumeli tomu, ktoré oblasti náš obrázok obsadzuje v priestore farieb, môžeme si zobraziť jednotlivé pixely v troch rovinách: v rovinách červená vs. zelená, červená vs. modrá a zelená vs. modrá. Každý bod pritom zafarbíme samotnou RGB farbou.

Ako vidno, naše farby skutočne obsadzujú len relatívne malý podpriestor priestoru farieb takže určitá kompresia by mala byť možná.



In [ ]:
np.random.seed(10)
sel_colors = colors[np.random.randint(0, len(colors), size=2500)]
plot_colors(sel_colors)

### Minidávkový $k$-means

Ďalej aplikujeme metódu $k$-means. Počet zhlukov nastavíme na 32 – to znamená, že budeme hľadať paletu s 32 farbami. Počet bodov, s ktorým budeme pracovať, je pomerne veľký. Z tohto dôvodu budeme používať minidávkovú verziu $k$-means – nájdenie stredov zhlukov to značne urýchli.

Myšlienka minidávkového $k$-means algoritmu spočíva v tom, že sa v každom kroku nepoužijú všetky body, ale sa z nich vyberie v každom kroku len podmnožine a pracuje sa s tou. Týmto spôsobom je možné aplikovať $k$-means dokonca aj na dáta, ktoré sa nezmestia celé naraz do pamäte. Stojí za povšimnutie, že ten istý princíp sa používa pri tréningu umelých neurónových sietí na veľmi veľkých dátových množinách.



In [ ]:
model = MiniBatchKMeans(n_clusters=32)
model.fit(X)

Keď sme model natrénovali, spustíme ho teraz na našej dátovej množine, čo nám ku každému bodu priradí identifikátor zhluku – ten určuje, ktorú farbu z palety priradíme danému pixelu. Takisto si extrahujeme aj samotnú paletu – skopírujeme si stredy zhlukov z modelu a transformujeme ich späť na 8-bitové celé čísla. Keď máme tieto dva prvky, náš obrázok je už vlastne kvantizovaný.



In [ ]:
clusts = model.predict(X)
cluster_centers = model.cluster_centers_.astype(np.uint8)

Aby sme si výsledky skontrolovali, zobrazíme si znovu všetky farby v troch rôznych rovinách a teraz si v tom istom grafe budeme vizualizovať aj stredy identifikovaných zhlukov.



In [ ]:
plot_colors(sel_colors, cluster_centers)

### Kvantizácia farieb obrázka

Napokon teda skúsme rekonštruovať obrázok z našej kvantizovanej verzie a pozrime sa, ako bude vyzerať výsledok. Jediná vec, ktorú treba spraviť, je prejsť znovu postupne všetky body a vyčítať z palety zodpovedajúce farby. Akonáhle ich máme k dispozícii, zmeníme tvar výslednej matice späť do tvaru pôvodného obrázka a takto rekonštruovaní obrázok zobrazíme.

Ako vidno, farby obrázka sú nepochybne menej živé, ale aj s 32 farbami sa nám podarilo charakter väčšej časti obrázka zachovať pomerne dobre. Najzrejmejšou výnimkou je v tomto prípade obloha, ktorú pôvodne tvoril gradient farieb a tento je teraz viditeľne kvantizovaný.



In [ ]:
quantized_X = cluster_centers[clusts]
quantized_img = quantized_X.reshape(img_shape)

plt.figure(figsize=(14, 10))
plt.imshow(quantized_img)
plt.axis('off');

---
### Task: Zopakujte kvantizáciu pre iné veľkosti palety

**Spustite algoritmus znovu s inými veľkosťami palety, napr. so 16 farbami alebo so 64 farbami. Vykreslite výsledné rekonštrukcie.** 

---


In [ ]:

# ---


### Kvantizácia s náhodnou paletou

Teraz sa, pre porovnanie, pokúsime realizovať kvantizáciu farieb s náhodne vygenerovanou paletou. Najprv si vyberieme náhodným spôsobom vopred stanovený počet farieb a následne nájdeme pre každý pixel v tejto palete najbližšiu farbu pomocou triedy `NearestNeighbors`. Napokon opäť zobrazíme výslednú rekonštrukciu. Ako uvidíte, výsledky v tomto prípade nie sú vôbec ideálne.



In [ ]:
np.random.seed(10)

model = NearestNeighbors(n_neighbors=1)
cluster_centers_ = np.random.uniform(0, 255, (32, 3))
model.fit(cluster_centers_)

clusts = model.kneighbors(X)[1]

cluster_centers = cluster_centers_.astype(np.uint8)
quantized_X = cluster_centers[clusts]
quantized_img = quantized_X.reshape(img_shape)

In [ ]:
plt.figure(figsize=(10, 6))
plt.imshow(quantized_img)
plt.axis('off');